In [2]:
pip install langchain


  Using cached langchain-0.2.17-py3-none-any.whl.metadata (7.1 kB)
  Using cached PyYAML-6.0.2-cp38-cp38-win_amd64.whl.metadata (2.1 kB)
  Using cached sqlalchemy-2.0.41-cp38-cp38-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.10.11-cp38-cp38-win_amd64.whl.metadata (8.0 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs

In [4]:
pip install -U langchain-community

  Using cached langchain_community-0.2.19-py3-none-any.whl.metadata (2.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.19-py3-none-any.whl (2.3 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.22.0-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pinecone,

SyntaxError: invalid syntax (2466217987.py, line 1)

In [14]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [26]:
PINECONE_API_KEY="pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2"
PINECONE_API_ENV="quickstart"

import os
from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2'



pc = pinecone.Pinecone(
    api_key= 'pcsk_4p2sTP_2S1sJULSLdTQsbXu1SDX3W6nihm7SuVHyMHu7cLzUKx4sroB2JYi4qPf6KK8Jf2',
)

if 'llama' not in pc.list_indexes().names():
   pc.create_index(
            name='llama',
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

In [31]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [32]:
#ectract data from pdf
def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [33]:
extracted_data = load_pdf("data/")

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extraxted_data)
print("length of my chunks", len(text_chunks))

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="mchat"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""